In [1]:
import pandas as pd

df= pd.read_csv("ecommerce_dataset.csv", names=["category", "description"], header=None)
print(df.shape)
df.head(3)

(50425, 2)


,category,description
0,Household,Paper Plane Design Framed Wall Hanging Motivat...
1,Household,"SAF 'Floral' Framed Painting (Wood, 30 inch x ..."
2,Household,SAF 'UV Textured Modern Art Print Framed' Pain...


In [2]:
df.dropna(inplace=True)
df.shape

(50424, 2)

In [3]:
df.category.unique()

array(['Household', 'Books', 'Clothing & Accessories', 'Electronics'],
      dtype=object)

In [4]:
df.category.replace("Clothing & Accessories", "Clothing_Accessories", inplace=True)

In [5]:
df.category.unique()

array(['Household', 'Books', 'Clothing_Accessories', 'Electronics'],
      dtype=object)

In [ ]:
When you train a fasttext model, it expects labels to be specified with __label__ prefix. We will just create a third column in the dataframe that has __label__ as well as the product description

In [7]:
df['category'] = '__label__' + df['category'].astype(str)
df.head(5)

,category,description
0,__label__Household,Paper Plane Design Framed Wall Hanging Motivat...
1,__label__Household,"SAF 'Floral' Framed Painting (Wood, 30 inch x ..."
2,__label__Household,SAF 'UV Textured Modern Art Print Framed' Pain...
3,__label__Household,"SAF Flower Print Framed Painting (Synthetic, 1..."
4,__label__Household,Incredible Gifts India Wooden Happy Birthday U...


In [8]:
df['category_description'] = df['category'] + ' ' + df['description']
df.head(3)

,category,description,category_description
0,__label__Household,Paper Plane Design Framed Wall Hanging Motivat...,__label__Household Paper Plane Design Framed W...
1,__label__Household,"SAF 'Floral' Framed Painting (Wood, 30 inch x ...",__label__Household SAF 'Floral' Framed Paintin...
2,__label__Household,SAF 'UV Textured Modern Art Print Framed' Pain...,__label__Household SAF 'UV Textured Modern Art...


In [ ]:
**Pre-procesing**
1. Remove punctuation
2. Remove extra space
3. Make the entire sentence lower case

In [9]:
import re

text = "  VIKI's | Bookcase/Bookshelf (3-Shelf/Shelve, White) | ? . hi"
text = re.sub(r'[^\w\s\']',' ', text)
text = re.sub(' +', ' ', text)
text.strip().lower()

"viki's bookcase bookshelf 3 shelf shelve white hi"

In [10]:
def preprocess(text):
    text = re.sub(r'[^\w\s\']',' ', text)
    text = re.sub(' +', ' ', text)
    return text.strip().lower() 

In [11]:
df['category_description'] = df['category_description'].map(preprocess)
df.head()

,category,description,category_description
0,__label__Household,Paper Plane Design Framed Wall Hanging Motivat...,__label__household paper plane design framed w...
1,__label__Household,"SAF 'Floral' Framed Painting (Wood, 30 inch x ...",__label__household saf 'floral' framed paintin...
2,__label__Household,SAF 'UV Textured Modern Art Print Framed' Pain...,__label__household saf 'uv textured modern art...
3,__label__Household,"SAF Flower Print Framed Painting (Synthetic, 1...",__label__household saf flower print framed pai...
4,__label__Household,Incredible Gifts India Wooden Happy Birthday U...,__label__household incredible gifts india wood...


In [14]:
#Train Test Split
from sklearn.model_selection import train_test_split

train, test = train_test_split(df, test_size=0.2)

In [15]:
train.shape, test.shape

((40339, 3), (10085, 3))

In [16]:
train.to_csv("ecommerce.train", columns=["category_description"], index=False, header=False)
test.to_csv("ecommerce.test", columns=["category_description"], index=False, header=False)

In [ ]:
#Train the model and evaluate performance

In [19]:
import fasttext

model = fasttext.train_supervised(input="ecommerce.train")
model.test("ecommerce.test")

(10084, 0.968167393891313, 0.968167393891313)

In [ ]:
First parameter (10084) is test size. Second and third parameters are precision and recall respectively. You can see we are getting around 96% precision which is pretty good

In [21]:
#Now let's do prediction for few product descriptions

model.predict("wintech assemble desktop pc cpu 500 gb sata hdd 4 gb ram intel c2d processor 3")

(('__label__electronics',), array([0.99024755]))

In [22]:
model.predict("ockey men's cotton t shirt fabric details 80 cotton 20 polyester super combed cotton rich fabric")

(('__label__clothing_accessories',), array([1.00001001]))

In [23]:
model.predict("think and grow rich deluxe edition")

(('__label__books',), array([1.00000989]))

In [24]:
model.get_nearest_neighbors("painting")

[(0.9990121126174927, 'tand'),
 (0.9990121126174927, 'zagmagatcleaning'),
 (0.9989484548568726, 'fulfilment'),
 (0.9989481568336487, "bud'"),
 (0.9989443421363831, '25b6'),
 (0.9989440441131592, 'fabir'),
 (0.9989440441131592, 'purpode'),
 (0.9989440441131592, 'katar'),
 (0.9989389181137085, 'x9'),
 (0.9989241361618042, 'clays')]

In [25]:
model.get_nearest_neighbors("sony")

[(0.9992609024047852, 'pict'),
 (0.9992608428001404, 'handycams'),
 (0.9992608428001404, '15mb'),
 (0.9992608428001404, 'mshx8b'),
 (0.9992597699165344, 'cd0051tx'),
 (0.9992597699165344, 'mx130discrete'),
 (0.9992512464523315, 'g42'),
 (0.9992473125457764, "network's"),
 (0.9992441534996033, '76mm'),
 (0.9992275834083557, 'droid')]

In [26]:
model.get_nearest_neighbors("banglore")

[(0.0, 'to'),
 (0.0, 'and'),
 (0.0, 'a'),
 (0.0, 'with'),
 (0.0, 'for'),
 (0.0, 'is'),
 (0.0, '</s>'),
 (0.0, 'f0bb00ydin'),
 (0.0, 'c2000'),
 (0.0, 'sleeves\xa0give')]